In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sympy as sp
from itertools import product

from Init import init_S, init_R, init_G
from Groups import GroupOhD 
from Functions import *
from Interpolators import momenta_1p,  momenta_2p, interpolator_1p, interpolator_2p

IRREPS  = {'2Oh' : ['G1g','G1u','Hg','Hu'],
           '2C4v': ['G1','G2'],
           '2C2v': ['G'],
           '2C3v': ['G','F1','F2']}
PREFS   = {'2Oh' : (0,0,0), 
           '2C4v': list(init_G['2C4v'].keys())[0], #0,0,1
           '2C2v': list(init_G['2C2v'].keys())[0], #0,1,1
           '2C3v': list(init_G['2C3v'].keys())[0],}#1,1,1

#initialize group
group_2Oh = GroupOhD()

#spin1212 matrix
S = extract_repr(group_2Oh,init_S)

#spin1 matrix
Rot3d = extract_repr(group_2Oh,init_R)

other_PREFS = { '2Oh' :{},
               '2C4v':{(0,0,-1):'\\bar{C}_2y',
                       (0,-1,0):'C_4x^i',
                       (0,1,0):'C_4x',
                       (1,0,0) :'C_4y^i',
                       (-1,0,0) :'C_4y', },
              '2C2v':{(0,1,-1):'C_4x^i',
                      (0,-1,1):'C_4x',
                      (0,-1,-1):'C_2f',
                      
                      (1,0,1) :'C_4z^i',
                      (-1,0,-1):'C_2b',
                      (1,0,-1):'C_2a',
                      (-1,0,1) :'C_4z',
                      
                      (1,1,0) :'C_4y',
                      (-1,-1,0) :'C_2d',
                      (1,-1,0):'C_2c',
                      (-1,1,0):'C_4y^i'},
              '2C3v':{(1,1,-1) :'C_4y',
                     (1,-1,1) :'C_4x',
                     (-1,1,1):'C_4y^i',
                     (1,-1,-1):'C_2x',
                     (-1,1,-1):'C_2y',
                     (-1,-1,1):'C_2z',
                     (-1,-1,-1):'C_2b',
                     }}


from pandas import read_pickle
coeff=read_pickle('coeffsN.pkl')
coeffNEW=read_pickle('NEWcoeffsN.pkl')

In [2]:
for group in coeff.keys():
    for irrep in coeff[group].keys():
        for l in coeff[group][irrep].keys():
            for mom in coeff[group][irrep][l].keys():
                for a in coeff[group][irrep][l][mom].keys():
                    for gamma in coeff[group][irrep][l][mom][a].keys():
                        continue
                        print(group,irrep,l,mom,a,gamma,coeff[group][irrep][l][mom][a][gamma].shape)

for group in coeffNEW.keys():
    for mom in coeffNEW[group].keys():
        for irrep in coeffNEW[group][mom].keys():
            for b in coeffNEW[group][mom][irrep].keys():
                for l in coeffNEW[group][mom][irrep][b].keys():
                    for a in coeffNEW[group][mom][irrep][b][l].keys():
                        for gamma in coeffNEW[group][mom][irrep][b][l][a].keys():
                            continue
                            print(group,mom,irrep,b,l,a,gamma,coeffNEW[group][mom][irrep][b][l][a][gamma].shape)

Structure of coeff: group,irrep,l,mom,a,gamma \
group = 2Oh,2C4v,2C2v,2C3v \
irrep = G1g,G1u,G1,G \
l = l1,l2 \
mom = up to N=3 \
occ = a,b \
gamma = cg5,cg5g4

Structure of coeffNEW: group,mom,irrep,b,l,a,gamma \
group,mom,irrep \
b = b1,b2 \
l,a \
gamma = gamma + c-g5, cg4-g5

In [5]:
coeff=read_pickle('coeffsN.pkl')
coeffNEW=read_pickle('NEWcoeffsN.pkl')
for group in coeff.keys():
    for irrep in coeff[group].keys():
        for l in coeff[group][irrep].keys():
            mom = {'2Oh':(0,0,0),'2C4v':(0,0,1),'2C2v':(0,1,1),'2C3v':(1,1,1)}[group]
            for a in coeff[group][irrep][l][mom].keys():
                for gamma in coeff[group][irrep][l][mom][a].keys():
                    t1=coeff[group][irrep][l][mom][a][gamma]
                    t2=coeffNEW[group][mom][irrep]['b1'][l][a][gamma]
                    if np.linalg.norm(t1 - t2) > 1e-5:
                        # if (np.linalg.norm(mom)>1):
                        #     continue
                        print(group,irrep,l,mom,a,gamma,t1,t2)

In [7]:
coeff=read_pickle('coeffsN.pkl')
coeffNEW=read_pickle('NEWcoeffsN.pkl')
for group in coeff.keys():
    for irrep in coeff[group].keys():
        for l in coeff[group][irrep].keys():
            for mom in coeff[group][irrep][l].keys():
                for a in coeff[group][irrep][l][mom].keys():
                    for gamma in coeff[group][irrep][l][mom][a].keys():
                        t1=coeff[group][irrep][l][mom][a][gamma]
                        t2=coeffNEW[group][mom][irrep]['b1'][l][a][gamma]
                        if np.linalg.norm(t1 - t2) > 1e-5:
                            print(group,irrep,l,mom,a,gamma)

In the case of ref moms, they agree

In [18]:
coeff=read_pickle('coeffsN.pkl')
coeffNEW=read_pickle('NEWcoeffsN.pkl')

gamma='cg5'
bases={};basesNEW={}
for group in coeff.keys():
    for irrep in coeff[group].keys():
        if irrep in ['G1u']:
            continue
        for l in coeff[group][irrep].keys():
            for mom in coeff[group][irrep][l].keys():
                if mom not in bases:
                    bases[mom]={}
                    basesNEW[mom]={}
                for a in coeff[group][irrep][l][mom].keys():
                    t1=coeff[group][irrep][l][mom][a][gamma]
                    t2=coeffNEW[group][mom][irrep]['b1'][l][a][gamma]
                    bases[mom][(l,a)]=coeff[group][irrep][l][mom][a][gamma]
                    basesNEW[mom][(l,a)]=coeffNEW[group][mom][irrep]['b1'][l][a][gamma]

for group in ['2C4v','2C2v','2C3v']:
    momRef=PREFS[group]
    for momCheck in other_PREFS[group].keys():
        print(group,momRef,momCheck)
        kys=list(bases[momRef].keys())
        t0=[bases[momRef][ky] for ky in kys]
        t=[bases[momCheck][ky] for ky in kys]
        res=np.linalg.inv(t0)@t
        for g in S:
            norm = np.abs(np.sum(np.abs(S[g]**-1-res)).evalf())
            if norm < 1e-2:
                print(g,norm)
        kys=list(basesNEW[momRef].keys())
        t0=[basesNEW[momRef][ky] for ky in kys]
        t=[basesNEW[momCheck][ky] for ky in kys]
        res=np.linalg.inv(t0)@t
        for g in S:
            norm = np.abs(np.sum(np.abs(S[g]**-1-res)).evalf())
            if norm < 1e-2:
                print(g,norm)
        print()

2C4v (0, 0, 1) (0, 0, -1)
C_2y 0
C_2y 0

2C4v (0, 0, 1) (0, -1, 0)
C_4x 3.86691732538117e-16
C_4x 3.86691732538117e-16

2C4v (0, 0, 1) (0, 1, 0)
C_4x^i 3.86691732538117e-16
C_4x^i 3.86691732538117e-16

2C4v (0, 0, 1) (1, 0, 0)
C_4y 1.27487015223824e-15
C_4y 1.27487015223824e-15

2C4v (0, 0, 1) (-1, 0, 0)
C_4y^i 1.27487015223824e-15
C_4y^i 1.27487015223824e-15

2C2v (0, 1, 1) (0, 1, -1)
C_4x^i 4.44089209850063e-16
C_4x^i 4.44089209850063e-16

2C2v (0, 1, 1) (0, -1, 1)
C_4x 4.44089209850063e-16
C_4x 4.44089209850063e-16

2C2v (0, 1, 1) (0, -1, -1)
C_2x 0
C_2x 0

2C2v (0, 1, 1) (1, 0, 1)
C_4z^i 3.67630733038497e-8
C_4z^i 3.67630733038497e-8

2C2v (0, 1, 1) (-1, 0, -1)
C_2b 3.94042043965486e-8
C_2b 3.94042043965486e-8

2C2v (0, 1, 1) (1, 0, -1)
C_2a 3.94042043965486e-8
C_2a 3.94042043965486e-8

2C2v (0, 1, 1) (-1, 0, 1)
C_4z 3.67630733038497e-8
C_4z 3.67630733038497e-8

2C2v (0, 1, 1) (1, 1, 0)
C_4y 4.44089209850063e-16
C_4y 4.44089209850063e-16

2C2v (0, 1, 1) (-1, -1, 0)
C_2d 4.440892098

In [9]:
coeff_rep=read_pickle('coeffsN_repro.pkl')
coeffNEW=read_pickle('NEWcoeffsN.pkl')
for group in coeff_rep.keys():
    for irrep in coeff_rep[group].keys():
        for l in coeff_rep[group][irrep].keys():
            for mom in coeff_rep[group][irrep][l].keys():
                for a in coeff_rep[group][irrep][l][mom].keys():
                    for gamma in coeff_rep[group][irrep][l][mom][a].keys():
                        t1=coeff_rep[group][irrep][l][mom][a][gamma]
                        t2=coeffNEW[group][mom][irrep]['b1'][l][a][gamma]
                        if np.linalg.norm(t1 - t2) > 1e-5:
                            print(group,irrep,l,mom,a,gamma)

# Npi

In [7]:
coeff=read_pickle('coeffsNpi.pkl')
coeffNEW=read_pickle('NEWcoeffsNpi.pkl')
for group in coeff.keys():
    for irrep in coeff[group].keys():
        for l in coeff[group][irrep].keys():
            for ptot in coeff[group][irrep][l].keys():
                for mom_Npi in coeff[group][irrep][l][ptot].keys():
                    for a in coeff[group][irrep][l][ptot][mom_Npi].keys():
                        for mom in coeff[group][irrep][l][ptot][mom_Npi][a].keys():
                            for gamma in coeff[group][irrep][l][ptot][mom_Npi][a][mom].keys():
                                continue
                                print(group,irrep,l,ptot,mom_Npi,a,mom,gamma,coeff[group][irrep][l][ptot][mom_Npi][a][mom][gamma].shape)

for group in coeffNEW.keys():
    for ptot in coeffNEW[group].keys():
        for mom_Npi in coeffNEW[group][ptot].keys():
            for irrep in coeffNEW[group][ptot][mom_Npi].keys():
                for b in coeffNEW[group][ptot][mom_Npi][irrep].keys():
                    for l in coeffNEW[group][ptot][mom_Npi][irrep][b].keys():
                        for a in coeffNEW[group][ptot][mom_Npi][irrep][b][l].keys():
                            for mom in coeffNEW[group][ptot][mom_Npi][irrep][b][l][a].keys():
                                for gamma in coeffNEW[group][ptot][mom_Npi][irrep][b][l][a][mom].keys():
                                    continue
                                    print(group,ptot,mom_Npi,irrep,b,l,a,mom,gamma,coeffNEW[group][ptot][mom_Npi][irrep][b][l][a][mom][gamma].shape)
                            

In [11]:
group='2Oh'; irrep='G1g'; ptot=(0,0,0); gamma='cg5;g5' 
l='l1'; mom_Npi='N1\\pi1'; a='a'; 

for mom in coeff[group][irrep][l][ptot][mom_Npi][a]:
    l='l1'
    print(mom,coeff[group][irrep][l][ptot][mom_Npi][a][mom][gamma])
    l='l2'
    print(mom,coeff[group][irrep][l][ptot][mom_Npi][a][mom][gamma])
    print()

((-1, 0, 0), (1, 0, 0)) [0.        +0.j 0.40824829+0.j 0.        +0.j 0.        +0.j]
((-1, 0, 0), (1, 0, 0)) [0.40824829+0.j 0.        +0.j 0.        +0.j 0.        +0.j]

((0, 0, -1), (0, 0, 1)) [0.40824829+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
((0, 0, -1), (0, 0, 1)) [ 0.        +0.j -0.40824829+0.j  0.        +0.j  0.        +0.j]

((0, 1, 0), (0, -1, 0)) [0.+0.j         0.+0.40824829j 0.+0.j         0.+0.j        ]
((0, 1, 0), (0, -1, 0)) [0.-0.40824829j 0.+0.j         0.+0.j         0.+0.j        ]

((1, 0, 0), (-1, 0, 0)) [ 0.        +0.j -0.40824829+0.j  0.        +0.j  0.        +0.j]
((1, 0, 0), (-1, 0, 0)) [-0.40824829+0.j  0.        +0.j  0.        +0.j  0.        +0.j]

((0, 0, 1), (0, 0, -1)) [-0.40824829+0.j  0.        +0.j  0.        +0.j  0.        +0.j]
((0, 0, 1), (0, 0, -1)) [0.        +0.j 0.40824829+0.j 0.        +0.j 0.        +0.j]

((0, -1, 0), (0, 1, 0)) [0.+0.j         0.-0.40824829j 0.+0.j         0.+0.j        ]
((0, -1, 0), (0, 1, 0)) [0.+0.408

In [4]:
for group in coeff.keys():
    for irrep in coeff[group].keys():
        for l in coeff[group][irrep].keys():
            for ptot in coeff[group][irrep][l].keys():
                for mom_Npi in coeff[group][irrep][l][ptot].keys():
                    for a in coeff[group][irrep][l][ptot][mom_Npi].keys():
                        for mom in coeff[group][irrep][l][ptot][mom_Npi][a].keys():
                            for gamma in coeff[group][irrep][l][ptot][mom_Npi][a][mom].keys():
                                try:
                                    t1=coeff[group][irrep][l][ptot][mom_Npi][a][mom][gamma]
                                    t2=coeffNEW[group][ptot][mom_Npi][irrep]['b1'][l][a][mom][gamma]
                                    if np.linalg.norm(t1 - t2) > 1e-5:
                                        print('bad',group,irrep,l,ptot,mom_Npi,a,mom,gamma)
                                    else:
                                        print('good:',group,irrep,l,ptot,mom_Npi,a,mom,gamma)
                                except Exception as e:
                                    print('Exception:',e)

AttributeError: 'numpy.ndarray' object has no attribute 'keys'